In [121]:
import os
import pandas as pd
path = 'classifier_cv_results/'
file_names = os.listdir(path)
[file_name for file_name in file_names if 'labels' in file_name]

['Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Marysville - All Excerpts - Final - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Newtown - All Excerpts - 1-2-2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Vegas - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Orlando - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'San Bernardino - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv']

## Sentence Based Results For Each Event

In [126]:
sentence_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        sentence_results_df = pd.concat([sentence_results_df, event_df], ignore_index=True)

Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv


In [127]:
df = sentence_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 
                                 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()

In [102]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [150]:
True&False

False

In [154]:
df = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 'precision', 
                                                                            'label', 'event',
                                                                            'num_test_exs', 'num_train_exs']]
labels = ['ALL']+list(set(df['label']))
events = ['ALL']+list(set(df['event']))
@interact
def show_results(column1='event', column2='label',
                 column3='num_test_exs', column4='num_train_exs',
                 event=events, label = labels, 
                 num_test = (0,50,1), num_train = (0,100,1)):
    
#     df_filt1 = df.loc[df[column3] > num_test_exs]
#     df_filt2 = df_filt1.loc[df_filt1[column4] > num_train_exs]
#     df = df_filt2
    df_len = df.shape[0]
    num_test_filt = df[column3] > num_test
    num_train_filt = df[column4] > num_train
    event_filt =[(df.loc[i, column1] == event) or (event == 'ALL') for i in range(df_len)]
    label_filt =[(df.loc[i, column2] == label) or (label == 'ALL') for i in range(df_len)]
    final_filt = [num_test_filt[i]&num_train_filt[i]&event_filt[i]&label_filt[i] for i in range(df_len)]
    
    return df.loc[final_filt]

interactive(children=(Text(value='event', description='column1'), Text(value='label', description='column2'), …

## Excerpt Based Results for Each Event

In [99]:
excerpt_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        excerpt_results_df = pd.concat([excerpt_results_df, event_df], ignore_index=True)

Marysville - All Excerpts - Final - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Newtown - All Excerpts - 1-2-2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Charleston - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Vegas - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Isla Vista - All Excerpts - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Orlando - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
San Bernardino - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv


In [100]:
df = excerpt_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()

In [120]:
df_ex = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 
                                                                            'precision', 'label', 'event']]
labels = ['ALL']+list(set(df_ex['label']))
events = ['ALL']+list(set(df_ex['event']))
@interact
def show_articles_for_event_and_label(column1='event', column2='label',
                            event=events, label = labels):
    if (event == 'ALL') and (label != 'ALL'):
        return df.loc[df_ex[column2] == label]
    elif (event != 'ALL') and (label == 'ALL'):
        return df.loc[df_ex[column1] == event]
    elif (event == 'ALL') and (label == 'ALL'):
        return df_ex
    else:
        return df_ex.loc[(df_ex[column1] == event) & (df_ex[column2] == label)]

interactive(children=(Text(value='event', description='column1'), Text(value='label', description='column2'), …

## All Events

### Sentence Based

In [52]:
sentence_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name and 'fulldata' in file_name:
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        sentence_results_df = pd.concat([sentence_results_df, event_df], ignore_index=True)

In [ ]:
df = sentence_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()
df_results = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 
                                                                            'precision', 'label', 'event']]
df_results

### Event Based

In [ ]:
event_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name and 'fulldata' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)      
        event_results_df = pd.concat([event_results_df, event_df], ignore_index=True)

In [ ]:
df = event_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()
df_results = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 
                                                                            'precision', 'label', 'event']]
df_results